<h1>Proyecto sobre interpretabilidad sobre modelos de caja negra</h1>

<p>En este proyecto nos dedicaremos a realizar el entrenamiento para los modelos de interpretabilidad de modelos de caja negra,en nuestro proyecto hemos realizado hacer el entrenamiento de modelos para tareas de clasifiacacion. Adelante en esta practica primero realizaremos el entrenamiento de los diferentes modelos de caja negra primero realizaremos el entrenamiento a un modelo de redes neuronales.</p>
<u><b>A continuacion el siguiente bloque de codigo es necesario ejecutarlo para el correcto funcionamiento de este notebook.</b></u>

In [126]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow import get_logger
get_logger().setLevel('ERROR')
import random
from tensorflow import random as tensorflow_random

random.seed(2398572)
tensorflow_random.set_seed(394867)

import pandas
from sklearn import model_selection

np.random.seed(43958734)
np.set_printoptions(threshold=10)
from tensorflow import keras as ke

from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist

<p>Ahora vamos a seleccionar el primer csv que contiene los datos y seleccionaremos los atributos que tendremos en cuenta para hacer el entrenamiento del algoritmo</p>

In [127]:
shill_bidding = pandas.read_csv("Shill Bidding Dataset.csv")
shill_bidding.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


Seleccionamos los atributos a tener encuenta a la hora de entrenar

In [128]:
atributos = shill_bidding.loc[:,'Bidder_Tendency':'Auction_Duration']
atributos = atributos.to_numpy()
print(atributos)


[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


<p>Vamos a seleccionar el atributo objetivo para el algoritmo</p>

In [129]:
objetivo = shill_bidding['Class']
objetivo = objetivo.to_numpy()
print(objetivo)

[0 0 0 ... 0 0 0]


Entrenamos el modelo de redes neuronales y dividimos el modelo de entrenamiento con el modelo de pruebas

In [130]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.4)

Input indica el numero de dimensiones y dense indica cantidad de neuronas y la funcion de activacion, que por defecto es la identidad. Este ejemplo es para modelos de <b>clasificacion</b> 

La red neuronal posee una única capa (aparte de la de entrada) que proporciona como salida un array bidimensional None x 1 (es decir, para cada ejemplo proporciona un valor) y 9 parámetros (el peso de la conexión de cada entrada con la neurona de la capa más el sesgo de la neurona) entrenables (la red los aprenderá mediante el algoritmo de entrenamiento).

In [131]:
red_bidding = ke.Sequential()
red_bidding.add(ke.Input(shape=(9,)))
red_bidding.add(ke.layers.Dense(10, activation='relu'))
red_bidding.add(ke.layers.Dense(1, activation='sigmoid'))

Los pesos y sesgos de la red se guardan en el atributo weights (son los arrays asociados al argumento numpy en la estructura de datos guardada en ese atributo).

In [132]:
red_bidding.summary()
red_bidding.weights

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                100       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense_8/kernel:0' shape=(9, 10) dtype=float32, numpy=
 array([[ 0.16066974, -0.1832436 , -0.3538186 , ...,  0.340855  ,
          0.25578886,  0.5128347 ],
        [-0.1894975 , -0.0651882 , -0.12789819, ..., -0.19957277,
         -0.0312627 , -0.17042843],
        [-0.4715354 , -0.20407248,  0.29305267, ..., -0.50047326,
          0.43153203,  0.34140068],
        ...,
        [-0.01709646, -0.03771824, -0.15322047, ...,  0.49320394,
          0.24029279, -0.05415136],
        [ 0.2125234 ,  0.41607064, -0.06427649, ..., -0.03796834,
          0.12062675,  0.19724232],
        [ 0.27244765,  0.48618132,  0.47343248, ...,  0.20641726,
          0.0910573 ,  0.37640864]], dtype=float32)>,
 <tf.Variable 'dense_8/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_9/kernel:0' shape=(10, 1) dtype=float32, numpy=
 array([[ 0.57531995],
        [ 0.17448515],
        [-0.48876932],
        [ 0.38587934]

Entrenamos la red neuronal con un optimizador y una funcion de perdida, en este caso al tratarse de un modelo de clasifiacion seleccionamos la funcion de perdida como la entropia cruzada binaria

In [133]:
red_bidding.compile(optimizer='SGD', loss='binary_crossentropy',
                   metrics=['accuracy'])

Aqui entrenamos el modelo con los atributos de prueba seleccionados y el objetivo de entrenamiento

In [134]:
red_bidding.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=10)

Epoch 1/10
380/380 [==============================] - 1s 1ms/step - loss: 0.4157 - accuracy: 0.8700
Epoch 2/10
380/380 [==============================] - 1s 1ms/step - loss: 0.3335 - accuracy: 0.8974
Epoch 3/10
380/380 [==============================] - 0s 1ms/step - loss: 0.2838 - accuracy: 0.8990
Epoch 4/10
380/380 [==============================] - 0s 1ms/step - loss: 0.2324 - accuracy: 0.9146
Epoch 5/10
380/380 [==============================] - 0s 1ms/step - loss: 0.1843 - accuracy: 0.9407
Epoch 6/10
380/380 [==============================] - 0s 1ms/step - loss: 0.1456 - accuracy: 0.9570
Epoch 7/10
380/380 [==============================] - 0s 1ms/step - loss: 0.1180 - accuracy: 0.9631
Epoch 8/10
380/380 [==============================] - 0s 1ms/step - loss: 0.0993 - accuracy: 0.9649
Epoch 9/10
380/380 [==============================] - 0s 1ms/step - loss: 0.0870 - accuracy: 0.9631
Epoch 10/10
380/380 [==============================] - 0s 1ms/step - loss: 0.0783 - accuracy: 0.9694

Y ahora vamos a evaluar el modelo y ejecutar su prediccion con los atributos de prueba

In [135]:
red_bidding.evaluate(atributos_prueba, objetivo_prueba)
red_bidding.predict(atributos_prueba)
j = 0
vp=0
vn=0
fp=0
fn=0
for i in red_bidding.predict(atributos_prueba):
    
    if(i > 0.5):
        i = 1
    elif(i <= 0.5):
        i = 0
    if(i == objetivo_prueba[j] and i == 1):
        vp+=1
    elif(i !=objetivo_prueba[j] and i == 1):
        fp +=1
    elif(i ==objetivo_prueba[j] and i == 0):
        vn +=1
    elif(i !=objetivo_prueba[j] and i == 0):
        fn +=1
    j+=1
    
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))

80/80 [==============================] - 0s 1ms/step
Buenos resultados: 2445
Malos resultados: 84
La precision es:  0.9466666666666667


In [136]:
wine_white = pandas.read_csv('winequality-white.csv', delimiter=";")
wine_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1001.0000,3.00,0.45,8.8,bueno
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,bueno
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,bueno
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,bueno
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,bueno


In [137]:
atributos1 = wine_white[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
atributos1 = atributos1.to_numpy()

objetivo = wine_white[['quality']]
objetivo = pandas.get_dummies(objetivo)
print(objetivo)
objetivo = objetivo.to_numpy().astype(float)

(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos1, objetivo, test_size=.4)

      quality_bueno  quality_decente  quality_excelente  quality_malo  \
0                 1                0                  0             0   
1                 1                0                  0             0   
2                 1                0                  0             0   
3                 1                0                  0             0   
4                 1                0                  0             0   
...             ...              ...                ...           ...   
4893              1                0                  0             0   
4894              0                1                  0             0   
4895              1                0                  0             0   
4896              0                0                  0             0   
4897              1                0                  0             0   

      quality_muy bueno  quality_perfecto  quality_regular  
0                     0                 0                0  
1

In [138]:
red_vino = ke.Sequential()
red_vino.add(ke.Input(shape=(11,)))
red_vino.add(ke.layers.Dense(10, activation='relu'))
# Aquie tantas neuronas como posibles clases
red_vino.add(ke.layers.Dense(7, activation='softmax'))

# Softmax es mejor activacion en clasificacion multiclase debido a que las
# probabilidades de todas las clases sumadas dan 1


In [139]:
red_vino.summary()
red_vino.weights

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 10)                120       
                                                                 
 dense_11 (Dense)            (None, 7)                 77        
                                                                 
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense_10/kernel:0' shape=(11, 10) dtype=float32, numpy=
 array([[ 0.4539268 , -0.07710254,  0.15135956, ..., -0.16307437,
          0.01278824,  0.11068517],
        [ 0.05725986,  0.23043233, -0.40509826, ...,  0.21256512,
         -0.00953633, -0.43251088],
        [ 0.29988438, -0.3497378 ,  0.30682307, ..., -0.19849265,
         -0.49182558,  0.01923186],
        ...,
        [ 0.4283719 , -0.33114657, -0.39846885, ..., -0.27326483,
         -0.18029127,  0.5209586 ],
        [-0.38899636,  0.22119498,  0.16854131, ..., -0.24828452,
          0.11630654,  0.33638996],
        [-0.20374537,  0.20948946, -0.01595587, ..., -0.14216772,
         -0.21667987,  0.08175027]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_11/kernel:0' shape=(10, 7) dtype=float32, numpy=
 array([[-0.2849492 ,  0.10531801,  0.44416118, ...,  0.08467072,
         -0.23218483

In [140]:
import tensorflow as tf
red_vino.compile( optimizer='SGD', loss='categorical_crossentropy',
                   metrics=['accuracy'])


red_vino.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=10)

red_vino.evaluate(atributos_prueba, objetivo_prueba)
predictions = red_vino.predict(atributos_prueba)
# Esto lo realizamos para poder visualizar una aproximacion
for prediction, true_label in zip(predictions, objetivo_prueba):
    print("Prediccion: ", prediction, "Etiqueta real:", true_label)
    
# Realizar un algoritm que coja el indice del maximo de la lista y que devuelva un 1 en ese lugar

Epoch 1/10
294/294 [==============================] - 1s 1ms/step - loss: 6.3618 - accuracy: 0.4329
Epoch 2/10
294/294 [==============================] - 0s 1ms/step - loss: 1.5246 - accuracy: 0.4435
Epoch 3/10
294/294 [==============================] - 0s 1ms/step - loss: 1.4309 - accuracy: 0.4435
Epoch 4/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3867 - accuracy: 0.4435
Epoch 5/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3619 - accuracy: 0.4435
Epoch 6/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3464 - accuracy: 0.4435
Epoch 7/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3361 - accuracy: 0.4435
Epoch 8/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3287 - accuracy: 0.4435
Epoch 9/10
294/294 [==============================] - 0s 1ms/step - loss: 1.3234 - accuracy: 0.4435
Epoch 10/10
62/62 [==============================] - 0s 1ms/step
Prediccion:  [0.43108433 0.29423022

Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.

Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.

 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423

 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423

Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 1. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [0. 0. 0. 0. 1. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.03330791] Etiqueta real: [1. 0. 0. 0. 0. 0. 0.]
Prediccion:  [0.43108433 0.29423022 0.0384247  0.02041457 0.16322856 0.01930968
 0.

# El siguiente modelo a realizar es random forest (ChatGPT dice que es el mas sencillo)

In [141]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [142]:
# Cargar el archivo CSV
data = pandas.read_csv('Shill Bidding Dataset.csv')

# Separar los atributos (X) y el objetio (y)
# Ajusta 'etiqueta' al nombre de la columna objetivo
X = data[['Bidder_Tendency','Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding','Auction_Bids', 'Starting_Price_Average', 'Early_Bidding','Winning_Ratio', 'Auction_Duration']]
X = X.to_numpy()
y = data['Class']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# n_stimators = numero de arboles y max_depth profundidad maxima de cada arbol 
randomForest_Bidding = RandomForestClassifier(n_estimators=100, max_depth=10)
print(X)

[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


In [143]:
# Entrenamos el modelo con los datos
randomForest_Bidding.fit(X_train, y_train )

RandomForestClassifier(max_depth=10)

In [144]:
predictions = randomForest_Bidding.predict(X_test)
# Esto lo realizamos para poder visualizar una aproximacion

vp=0
vn=0
fp=0
fn=0
for prediction, true_label in zip(predictions, y_test):
    if(prediction == true_label and prediction == 1  ):
        vp+=1
    elif(prediction != true_label and prediction == 1):
        fp +=1
    elif(prediction == true_label and prediction == 0):
        vn +=1
    elif(prediction != true_label and prediction == 0):
        fn +=1

        
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))


Buenos resultados: 2505
Malos resultados: 24
La precision es:  0.9446494464944649


In [145]:
# Cargar el archivo CSV
data = pandas.read_csv('winequality-white1.csv', delimiter=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [146]:

# Separar los atributos (X) y el objetio (y)
# Ajusta 'etiqueta' al nombre de la columna objetivo
X = data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
X = X.to_numpy()
y = data['quality']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# n_stimators = numero de arboles y max_depth profundidad maxima de cada arbol 
randomForest_vinos = RandomForestClassifier(n_estimators=100, max_depth=10)
print(X)

[[ 7.    0.27  0.36 ...  3.    0.45  8.8 ]
 [ 6.3   0.3   0.34 ...  3.3   0.49  9.5 ]
 [ 8.1   0.28  0.4  ...  3.26  0.44 10.1 ]
 ...
 [ 6.5   0.24  0.19 ...  2.99  0.46  9.4 ]
 [ 5.5   0.29  0.3  ...  3.34  0.38 12.8 ]
 [ 6.    0.21  0.38 ...  3.26  0.32 11.8 ]]


In [147]:
# Entrenamos el modelo con los datos
randomForest_vinos.fit(X_train, y_train )

RandomForestClassifier(max_depth=10)

In [148]:
predictions = randomForest_vinos.predict(X_test)
# Esto lo realizamos para poder visualizar una aproximacion
for prediction, true_label in zip(predictions, y_test):
    print("Prediccion: ", prediction, "Etiqueta real:", true_label)


Prediccion:  7 Etiqueta real: 7
Prediccion:  8 Etiqueta real: 8
Prediccion:  6 Etiqueta real: 8
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 4
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 6
Prediccion:  6 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 8
Prediccion:  6 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 6
Prediccion:  6 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 8
Prediccion:  5 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 6
Prediccion:  7 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Predicci

# Este es el modelo LIME de interpretabilidad de modelos de caja negra

Este es el modelo de Lime que funciona podemos modificarlo averiguando la distancia con otro calculo como el de arriba y haciendo llamdas recursivas a funciones auxiliares

In [149]:
#Version 1
def perturb(datos, cabecera):
    k = random.randint(1, len(datos))
    datos_perturbados = datos.copy()
    for j in range(k):
        atributo_a_perturbar = random.randint(0,len(datos)-1)
        rango_maximo = abs(max(a[atributo_a_perturbar] for a in cabecera))
        rango_minimo = abs(min(a[atributo_a_perturbar] for a in cabecera))
        datos_perturbados[atributo_a_perturbar] = random.uniform(rango_minimo, rango_maximo)
    return datos_perturbados

def explicacion_del_modelo(N,f,datos, cabecera):
    X = [] # Muestras perturbadas
    R = [] # Representaciones
    W = [] # Distancias entre muestra x y sus perturbaciones
    for i in range(N):
        datos_perturbados= perturb(datos, cabecera)
        w = abs(sum(datos[atributo] - datos_perturbados[atributo] for atributo in range(0, len(datos)-1)))
        r = [0 if datos_perturbados[atributo] == datos[atributo] else 1 for atributo in range(0,len(datos)-1)]
        X.append(datos_perturbados)
        R.append(r)
        W.append(w)
    X = np.expand_dims(datos_perturbados, axis=0)
    Y_perturbada = f.predict(X)
    G = Ridge()
    G.fit(R, Y_perturbada, sample_weight=W)
    return G.coef_


In [150]:
#Version 2
def explicacion_del_modelo(N,f,datos, cabecera):
    X = [] # Muestras perturbadas
    R = [] # Representaciones
    W = [] # Distancias entre muestra x y sus perturbaciones
    for i in range(N):
        datos_perturbados= perturb(datos, cabecera)
        w = abs(sum(datos[atributo] - datos_perturbados[atributo] for atributo in range(0, len(datos)-1)))
        r = [0 if datos_perturbados[atributo] == datos[atributo] else 1 for atributo in range(0,len(datos)-1)]
        X.append(datos_perturbados)
        R.append(r)
        W.append(w)
    X = np.expand_dims(datos_perturbados, axis=0)
    Y_perturbada = f.predict(X)
    G = Ridge()
    G.fit(R, Y_perturbada, sample_weight=W)
    pesos = G.coef_
    # Paso 4: Generación de la explicación
    return pesos

In [158]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

# Ejemplo utilizando el modelo "red_bidding" definido en el código anterior
parametros = explicacion_del_modelo(1, red_bidding, atributos[5809], atributos)
params = explicacion_del_modelo(1, randomForest_Bidding, atributos[5809], atributos)
print("Parametros de la 1ª muestra:")
G = Ridge()
Y_perturbada = red_bidding.predict(atributos)
G.fit(atributos, Y_perturbada)
pesos = G.coef_
    
print(pesos)
print("HASTA AQUI HEMOS LLEGAO")
print(parametros)
print("--------------------------------")
G = Ridge()
Y_perturbada = randomForest_Bidding.predict(atributos)
G.fit(atributos, Y_perturbada)
pesos = G.coef_
   
print(pesos)
print("HASTA AQUI HEMOS LLEGAO")
print(params)
params = explicacion_del_modelo(1, randomForest_vinos, atributos1[4809], atributos1)
parametros = explicacion_del_modelo(1, red_vino, atributos1[4809], atributos1)
print("Parametros de la 2ª muestra:")
G = Ridge()
Y_perturbada = randomForest_vinos.predict(atributos1)
G.fit(atributos1, Y_perturbada)
pesos = G.coef_
    
print(pesos)
print("HASTA AQUI HEMOS LLEGAO")
print(parametros)
print("--------------------------------")
G = Ridge()
Y_perturbada = red_vino.predict(atributos1)
G.fit(atributos1, Y_perturbada)
pesos = G.coef_
   
print(pesos)
print("HASTA AQUI HEMOS LLEGAO")
print(params)

1/1 [==============================] - 0s 21ms/step
Parametros de la 1ª muestra:
198/198 [==============================] - 0s 929us/step
[[ 5.45765375e-02  8.31658761e-02  7.79039334e-01 -9.39381838e-04
   3.33292991e-03 -9.05059092e-03  1.57693132e-04  1.75979046e-02
  -6.44392750e-03]]
HASTA AQUI HEMOS LLEGAO
[[0. 0. 0. 0. 0. 0. 0. 0.]]
--------------------------------
[-0.0044386   0.01395906  0.96092548  0.03523234  0.004983    0.00990367
 -0.02463273  0.04984225  0.00218953]
HASTA AQUI HEMOS LLEGAO
[0. 0. 0. 0. 0. 0. 0. 0.]
1/1 [==============================] - 0s 23ms/step
Parametros de la 2ª muestra:
[-5.14681057e-02 -1.17707193e+00 -1.71168675e-02 ...  3.24291143e-01
  5.25725217e-01  4.42595843e-05]
HASTA AQUI HEMOS LLEGAO
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
--------------------------------
154/154 [==============================] - 0s 948us/step
[[ 0.00000

<p><strong>Alpha:</strong> Este parámetro controla la fuerza de la regularización en el modelo Ridge. Cuanto mayor sea el valor de alpha, mayor será la penalización aplicada a los coeficientes. Un valor de alpha igual a 1.0 indica una regularización moderada.</p>

<p><strong>Copy_X:</strong> Es un parámetro booleano que indica si los datos de entrada deben ser copiados antes de ajustar el modelo. Si se establece en True, se realiza una copia de los datos; si se establece en False, los datos se sobrescriben.</p>

<p><strong>Fit_intercept:</strong> Es un parámetro booleano que indica si se debe ajustar un intercepto en el modelo. Si se establece en True, el modelo incluirá un término constante; si se establece en False, el modelo no incluirá un término constante.</p>

<p><strong>Max_iter:</strong> Es el número máximo de iteraciones permitidas durante el ajuste del modelo. Si se establece en None, el modelo ajustará hasta que la convergencia sea alcanzada.</p>

<p><strong>Positive:</strong> Es un parámetro booleano que indica si los coeficientes deben ser no negativos. Si se establece en True, se restringe que los coeficientes sean siempre positivos.</p>

<p><strong>Random_state:</strong> Este parámetro controla la semilla utilizada por el generador de números aleatorios. Si se establece en None, el generador de números aleatorios utilizará una semilla diferente en cada ejecución.</p>

<p><strong>Solver:</strong> Especifica el algoritmo a utilizar en el ajuste del modelo. Si se establece en 'auto', el modelo seleccionará automáticamente el mejor algoritmo según los datos y el problema en cuestión.</p>

<p><strong>Tol:</strong> Especifica la tolerancia para el criterio de parada en el ajuste del modelo. Si la diferencia entre dos iteraciones consecutivas es menor que la tolerancia, el ajuste se considera convergido.</p>

<p><strong>Coef_:</strong> Especifica el peso que en esta caso se trata de la importancia de cada atributo a la hora de tomar decisiones para clasificar las muestras</p>


# Aqui tenemos los calculos requeridos sobre el trabajo

 <h1>Resumen de cálculos</h1>

  <i><b>1. Identidad:</b></i>
  <p>Verifica si las explicaciones son idénticas para objetos idénticos.</p>

 <i><b>2. Separabilidad:</b></i>
  <p>Verifica que las explicaciones sean diferentes para objetos no idénticos.</p>

 <i><b>3. Estabilidad:</b></i>
  <p>Evalúa si objetos similares tienen explicaciones similares mediante la correlación de Spearman.</p>

  <i><b>4. Selectividad:</b></i>
  <p>Mide el efecto de eliminar variables relevantes en la predicción mediante el cálculo del área bajo la curva (AUC).</p>

  <i><b>5. Completitud (Coherencia):</b></i>
  <p>Calcula la coherencia de una muestra al eliminar características no importantes.</p>

  <i><b>6. Completitud (Coherencia / Explicación):</b></i>
  <p>Calcula la coherencia promedio y la desviación estándar de la coherencia.</p>

 <i><b>7. Completitud (Congruencia):</b></i>
  <p>Calcula la desviación estándar de la coherencia como un proxy de congruencia.</p>


In [115]:
def identidad_(dist_objetos, dist_explicaciones):
    identidad = np.all(dist_objetos == 0) and np.all(dist_explicaciones == 0)
    return identidad
    
def separabilidad_(dist_objetos, dist_explicaciones):
    separabilidad = np.all(dist_objetos != 0) and np.all(dist_explicaciones > 0)
    return separabilidad

def estabilidad_(dist_objetos, dist_explicaciones):
    correlacion = [spearmanr(dist_objetos[i], dist_explicaciones[i]).correlation for i in range(len(dist_objetos))]
    estabilidad = all(corr > 0 for corr in correlacion)
    return estabilidad

def selectividad_(muestras, n_caracteristicas, modelo):
    for i in range(n_caracteristicas):
        muestras_modificadas = np.copy(muestras)
        muestras_modificadas[:, i] = 0
        
        # Obtener las etiquetas binarias para la característica i
        y_true = np.where(muestras[:, i] == 0, 0, 1)
        y_true_binary = label_binarize(y_true, classes=np.unique(y_true))
        
        # Obtener las predicciones del modelo con la característica i eliminada
        y_score = modelo.predict(muestras_modificadas)
        
        # Calcular el área bajo la curva (AUC) y almacenarlo en auc_scores
        if len(np.unique(y_true_binary)) > 1:
            auc_scores.append(roc_auc_score(y_true_binary, y_score))
    
    if auc_scores:
        selectividad = np.mean(auc_scores)
    else:
        selectividad = 0.0
    
    return selectividad

def coherencia_(prediccion_original, prediccion_nueva):
    # Obtener las predicciones del modelo para la señal original y la nueva señal
    # Calcular el error de predicción para la señal original
    error_original = np.abs(predicciones_original - prediccion_nueva)
    # Calcular el error de predicción para la nueva señal
    error_nueva = np.abs(predicciones_nueva - y_nueva)
    coherencia = np.abs(prediction_original - prediction_nueva)
    return coherencia
def completitud_(prediccion_original, y_test):
    pi_e= np.mean(prediccion_original != y_test)
    ei_e = np.mean(np.abs(prediccion_original - y_original))
    completitud = np.abs(ei_e / pi_e) 
    return completitud
    
def congruencia_(coherencia):
    #Congruencia
    alpha = np.mean(completeness)  # alpha_values es una lista/array con los valores de coherencia alpha_i

    # Calcular la desviación estándar de la coherencia
    N = len(data)
    delta = np.sqrt(np.mean((completeness - alpha)**2) / N)  # N es el número total de muestras
    return delta